### Raw Data Download Notebook

Zali et al. 2024 pulls horizontal seismic data from a single horizontal component from a single station, demeaned and detrended but not converted from instrument response. The data is pulled from March 12 to June 24, ~104 days, beginning 7 days before the Geldingadalir eruption began. The data is also downsampled to the minimum sampling rate necessary to observe the local volcanic tremor.

*Important Note:* This notebook runs using a different environment than this project's computational notebook. ObsPy is required, along with numpy and matplotlib

This notebook will download 100 days of data, beginning on March 1st, 2005 and ending June 9th, 2005, from sensor CC.STD of the Cascade Chain Volcano Monitoring network, located on Mount St. Helens on Studebaker Ridge. Mount St. Helens had a series of eruptions of ash and steam between 2004 and 2008, but this will focus on the second explosive eruption on March 8th, 2005 that was preceeded by seismic activity. https://www.usgs.gov/volcanoes/mount-st.-helens/science/2004-2008-renewed-volcanic-activity


In [2]:
!pip install obspy

  Using cached obspy-1.4.1-cp311-cp311-macosx_10_13_x86_64.whl
  Using cached scipy-1.14.1-cp311-cp311-macosx_14_0_x86_64.whl.metadata (60 kB)
  Using cached matplotlib-3.9.2-cp311-cp311-macosx_10_12_x86_64.whl.metadata (11 kB)
  Using cached lxml-5.3.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (3.8 kB)
  Using cached SQLAlchemy-1.4.54-cp311-cp311-macosx_10_9_universal2.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached contourpy-1.3.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.54.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (163 kB)
  Using cached kiwisolver-1.4.7-cp311-cp311-macosx_10_9_x86_64.whl.metadata (6.3 kB)
  Using cached pillow-11.0.0-cp311-cp311-macosx_10_10_x86_64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached greenlet-3.1.1-cp311-cp311-macosx_11_0_universal2.wh

In [3]:
import obspy
from obspy import UTCDateTime as utc
from obspy.clients.fdsn import Client

import numpy as np
import os

### Data Description

The seismic data is downloaded from the IRIS/Earthscope Database using ObsPy. The data downloads as a trace, an array of data with attached metadata, which is then packaged into a stream, which can contain multiple traces. The daily seismic data records are then saved as [mseed files](https://ds.iris.edu/ds/nodes/dmc/data/formats/miniseed/), which preserve this data+metadata structure, but requires ObsPy or other specialized software to open.

Set all of your variables for the station:

In [4]:
client = Client('IRIS')

#creating variables to download data
starttime = utc('2005-03-01T00:00:00')
endtime = starttime + 100 * (60*60*24)
day_range = 100

#also add a buffer to both ends to chop off once the data has been filtered
#and downsampled, kind of arbitrary length, 5% of a day (default ObsPy taper length)
buffer = 60*60*24*0.05 #seconds

net = 'CC'
sta = 'STD'
loc = '*' #wildcard, generally don't care about location code
cha = 'BHN' #horizontal component, as used in Zali et al

In [5]:
#create folder for numpy streams to go into and initialize filepath
!mkdir data
!mkdir data/raw
filepath = os.getcwd() + '/data/raw/'

#create arrays to save dates
dates = np.array([])

print(f'Downloading data from {starttime} to {endtime}')

#download the data piecemeal, here by day
for day in range(day_range):
    
    tr_length = 24*60*60
    
    # Format the current date as YYYYMMDD for the filename
    date_str = starttime.strftime('%Y%m%d')
    filename = f"{day + 1}_{date_str}_{net}{sta}.mseed"
    
    # if file already exists, then skip
    if os.path.exists(filepath + filename,):
        print(f'Data for day {date_str} already exists, skipping')
        starttime += tr_length
    else:
        try:
            # actually downloading
            st = client.get_waveforms(network=net,
                                    station=sta,
                                    location=loc,
                                    channel=cha,
                                    starttime=starttime-buffer,
                                    endtime=starttime+buffer+tr_length)

            # instrument sampling rate (hz)
            freq = st[0].stats.sampling_rate

            # merge traces within stream, linearly interpolating any gaps
            st.merge(fill_value='interpolate')

            ## Save data as MSEED, standard for storing seismic data
            st.write(filepath + filename, format='MSEED')

            # adding date
            dates = np.append(dates, starttime.date)

            print(f'Downloaded day {day + 1}')
        except Exception as e:
            print(f'Error downloading data for day {day + 1}: {e}')
        
    starttime += tr_length

#save dates list for future use
np.save(filepath+'date_list.csv', dates)

print(f'data download complete, saved to {filepath}')

Downloaded day 1
Downloaded day 2
Downloaded day 3
Downloaded day 4
Downloaded day 5
Downloaded day 6
Downloaded day 7
Downloaded day 8
Downloaded day 9
Downloaded day 10
Downloaded day 11
Downloaded day 12
Downloaded day 13
Downloaded day 14
Downloaded day 15
Downloaded day 16
Downloaded day 17
Downloaded day 18
Downloaded day 19
Downloaded day 20
Downloaded day 21
Downloaded day 22
Downloaded day 23
Downloaded day 24
Downloaded day 25
Downloaded day 26
Downloaded day 27
Downloaded day 28
Error downloading data for day 29: No data available for request.
HTTP Status code: 204
Detailed response of server:


Error downloading data for day 30: No data available for request.
HTTP Status code: 204
Detailed response of server:


Error downloading data for day 31: No data available for request.
HTTP Status code: 204
Detailed response of server:


Error downloading data for day 32: No data available for request.
HTTP Status code: 204
Detailed response of server:


Error downloading data for da

### Data Modalities and Formats

#### Data Modalities
The dataset consists of seismic data collected from a single horizontal component (BHN) of the CC.STD. The data is recorded continuously over a period of 100 days, capturing the seismic activity before, during, and after the volcanic eruption.

#### Data Formats
1. **MSEED (Mini-SEED) Files**:
    - The seismic data is stored in Mini-SEED format, which is a compact binary format used for storing time series data. Each file contains a day's worth of seismic data, including metadata such as the sampling rate and station information.
    - Example file name: `1_YYYYMMDD_hvuwb.mseed`, `2_YYYYMMDD_hwuwb.mseed`, ..., `60_YYYYMMDD_hvuwb.mseed`.
2. **Numpy Arrays**:
    - The dates corresponding to each day's seismic data are stored in a numpy array and saved as a CSV file (`date_list.csv`). This array helps in mapping the MSEED files to their respective dates.
    - Example: `dates = np.array([datetime.date(2021, 5, 16), datetime.date(2021, 5, 17), ...])`.

#### Data Processing
- The raw seismic data is downloaded using the ObsPy library from the IRIS/Earthscope Database.
- The data is merged and interpolated to fill any gaps, ensuring a continuous time series.
- The processed data is saved in the MSEED format, preserving both the time series and metadata.

This structured approach ensures that the seismic data is well-organized and easily accessible for further analysis and processing.

In [6]:
from obspy import read
import glob

# Folder containing mseed files
#folder_path = '/path/to/your/folder'

filepath = os.getcwd() + '/data/raw/'

# Get the first mseed file in the folder
first_file = glob.glob(f"{filepath}/*.mseed")[0]

# Read and display the file
st = read(first_file)
print(st)

1 Trace(s) in Stream:
CC.STD..BHN | 2005-04-09T22:48:00.000000Z - 2005-04-11T01:12:00.000000Z | 50.0 Hz, 4752001 samples
